# Kaggle Titanic

## 1. Setup

### 1.1 Load the libraries and the dataset

In [1]:
import numpy as np
import os
import pandas as pd

from IPython.display import display

In [2]:
train_url = "http://s3.amazonaws.com/assets.datacamp.com/course/Kaggle/train.csv"
test_url = "http://s3.amazonaws.com/assets.datacamp.com/course/Kaggle/test.csv"

train = pd.read_csv(train_url, index_col='PassengerId')
test = pd.read_csv(test_url, index_col='PassengerId')

### 1.2 A brief glance at the data

Let's take a look at the data

In [3]:
pd.DataFrame(
    np.vstack((train.shape, test.shape)),
    columns=['rows', 'columns'],
    index=['Training set', 'Test set']
)

,rows,columns
Training set,891,11
Test set,418,10


In [4]:
train.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [5]:
test.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,
892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [6]:
train.describe()

,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [7]:
test.describe()

,Pclass,Age,SibSp,Parch,Fare
count,418.000000,332.000000,418.000000,418.000000,417.000000
mean,2.265550,30.272590,0.447368,0.392344,35.627188
std,0.841838,14.181209,0.896760,0.981429,55.907576
min,1.000000,0.170000,0.000000,0.000000,0.000000
25%,1.000000,21.000000,0.000000,0.000000,7.895800
50%,3.000000,27.000000,0.000000,0.000000,14.454200
75%,3.000000,39.000000,1.000000,0.000000,31.500000
max,3.000000,76.000000,8.000000,9.000000,512.329200


In [8]:
train.dtypes

Survived      int64
Pclass        int64
Name         object
Sex          object
Age         float64
SibSp         int64
Parch         int64
Ticket       object
Fare        float64
Cabin        object
Embarked     object
dtype: object

In [9]:
test.dtypes

Pclass        int64
Name         object
Sex          object
Age         float64
SibSp         int64
Parch         int64
Ticket       object
Fare        float64
Cabin        object
Embarked     object
dtype: object

In [10]:
df = train.head()

## 1.3 Minor Cleanups

### 1.3.1 Split Name into First Name & Last Name Columns

In [11]:
def split_name(df):
    new_df = df['Name'].str.split(',', expand=True)
    new_df.columns=['Last Name', 'First Name']
    df.drop(columns=['Name'], inplace=True)
    return pd.concat((df, new_df), axis=1)

In [12]:
df_train = split_name(train)
df_test = split_name(test)

#### 1.3.2 One Hot Encoding

In [13]:
def one_hot_encode(df):
    df['Sex'] = (df['Sex'] == 'female')
    em = pd.get_dummies(df['Embarked']).rename(columns={
        'C': 'Embarked_C',
        'S': 'Embarked_S'
    })
    df = pd.concat((df.drop(columns=['Embarked']), em), axis=1)
    return df

In [14]:
df_train2 = one_hot_encode(df_train)
df_test2 = one_hot_encode(df_test)

#### 1.3.3 Get X & y

In [15]:
def get_xy(df, target_col):
    feature_cols = [x for x in df.columns if x != target_col]
    X = df[feature_cols]._get_numeric_data()
    y = df[target_col]
    return X.values, y.values

def get_x(df):
    return df._get_numeric_data().values

In [16]:
X_train, y_train = get_xy(df_train2.dropna(), 'Survived')
X_test = get_x(df_test)

## 2. Machine Learning

### 2.1 Logistic Regression

In [17]:
from sklearn.linear_model import LogisticRegression

In [18]:
model = LogisticRegression()
lr = model.fit(X_train, y_train)
y_pred = lr.predict(X_test)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').